In [41]:
import re
file = "./zigbee-herdsman-converters/src/devices/adurosmart.ts"
file_path = "./zigbee-herdsman-converters/src/devices/hzc_electric.ts"

with open(file_path, 'r') as file:
    content = file.read()

print(content)





import {Definition} from '../lib/types';
import * as exposes from '../lib/exposes';
import fz from '../converters/fromZigbee';
import {deviceEndpoints, light} from '../lib/modernExtend';
const e = exposes.presets;

const definitions: Definition[] = [
    {
        zigbeeModel: ['DimmerSwitch-2Gang-ZB3.0'],
        model: 'D086-ZG',
        vendor: 'HZC Electric',
        description: 'Zigbee dual dimmer',
        extend: [
            deviceEndpoints({endpoints: {'l1': 1, 'l2': 2}}),
            light({endpointNames: ['l1', 'l2'], configureReporting: true}),
        ],
    },
    {
        zigbeeModel: ['TempAndHumSensor-ZB3.0'],
        model: 'S093TH-ZG',
        vendor: 'HZC Electric',
        description: 'Temperature and humidity sensor',
        fromZigbee: [fz.temperature, fz.humidity, fz.linkquality_from_basic],
        toZigbee: [],
        exposes: [e.temperature(), e.humidity()], // Unfortunately, battery percentage is not reported by this device
    },
];

export default de

In [46]:
# Refining the approach to accurately capture and extract the `extend` part from each object

# Define a refined pattern to match objects including those with nested arrays or objects
object_pattern = r"\{[^{}]*?(extend: \[[\s\S]*?\])[^{}]*?\}"

# Extract all matches for objects containing an 'extend' part
extended_objects = re.findall(object_pattern, content)

# Extract the 'extend' parts from these matches
extend_functions = []
for obj in extended_objects:
    # Extracting function names within the extend part
    function_names = re.findall(r"(\w+)\(", obj)
    extend_functions.append(function_names)

extend_functions


[['deviceEndpoints', 'light']]

In [50]:
import re
import json

# Given string with the definitions
input_string = """
const definitions: Definition[] = [
    {
        zigbeeModel: ['DimmerSwitch-2Gang-ZB3.0'],
        model: 'D086-ZG',
        vendor: 'HZC Electric',
        description: 'Zigbee dual dimmer',
        extend: [
            deviceEndpoints({endpoints: {'l1': 1, 'l2': 2}}),
            light({endpointNames: ['l1', 'l2'], configureReporting: true}),
        ],
    },
    {
        zigbeeModel: ['TempAndHumSensor-ZB3.0'],
        model: 'S093TH-ZG',
        vendor: 'HZC Electric',
        description: 'Temperature and humidity sensor',
        fromZigbee: [fz.temperature, fz.humidity, fz.linkquality_from_basic],
        toZigbee: [],
        extend: [e.temperature(), e.humidity()],
        exposes: [e.temperature(), e.humidity()], // Unfortunately, battery percentage is not reported by this device
    },
];
"""

# Pattern to match the content of the definitions array
pattern = r"const definitions: Definition\[\] = (\[.*?\]);"

# Use DOTALL to match across multiple lines
matches = re.findall(pattern, input_string, re.DOTALL)

# Assuming we have matches and focusing on the extend case
if matches:
    # Extract the first match, which should be the content of the definitions array
    definitions_content = matches[0]

    # Define a refined pattern to match objects including those with nested arrays or objects
    object_pattern = r"\{[^{}]*?(extend: \[[\s\S]*?\])[^{}]*?\}"

    # Extract all matches for objects containing an 'extend' part
    extended_objects = re.findall(object_pattern, input_string)

    # Extract the 'extend' parts from these matches
    extend_functions = []
    for obj in extended_objects:
        print(obj)
        # Extracting function names within the extend part
        function_names = re.findall(r"(\w+)\(", obj)
        extend_functions.append(function_names)

extend_functions


extend: [
            deviceEndpoints({endpoints: {'l1': 1, 'l2': 2}}),
            light({endpointNames: ['l1', 'l2']


[['deviceEndpoints', 'light']]

In [45]:
import re

def extract_info(definitions_str):
    # Regex to match each definition block
    definition_blocks = re.findall(r'{[^}]*}', definitions_str)

    extracted_info = []
    for block in definition_blocks:
        # Extracting various fields using regular expressions
        zigbeeModel = re.findall(r"zigbeeModel: (\[[^\]]+\])", block)
        fromZigbee = re.findall(r"fromZigbee: (\[[^\]]*\])", block)
        toZigbee = re.findall(r"toZigbee: (\[[^\]]*\])", block)
        pattern = r"\{[^{}]*?(extend: \[[\s\S]*?\])[^{}]*?\}"
        print(block)
        extend_lines = re.findall(pattern, block)
        function_pattern = r'(\w+)\('
        extend = []
        print("Extend lines are: ", extend_lines)
        for line in extend_lines:
            print("Line is: ", line)
            function_calls = re.findall(function_pattern, line)
            for call in function_calls:
                extend.append(call)
        res = []
        
        for fz in fromZigbee:

            pattern = r'((\w+\.fz\.\w+)|(fz\.\w+))'
            fromZigbee = re.findall(pattern, fz)
            if fromZigbee:
                res.extend([_[0] for _ in fromZigbee])
        fromZigbee = res
        
        
        res = []
        
        for fz in toZigbee:

            pattern = r'((\w+\.tz\.\w+)|(tz\.\w+))'
            toZigbee = re.findall(pattern, fz)
            if toZigbee:
                res.extend([_[0] for _ in toZigbee])
        toZigbee = res
        # Converting extracted fields to their actual types (lists)
        # Using eval to convert string representations of lists to actual lists
        # Note: Using eval can be dangerous if you're processing untrusted input
        info = {
            "zigbeeModels": zigbeeModel,
            "fromZigbee": fromZigbee,
            "toZigbee": toZigbee,
            "extend": extend
        }
        if info["zigbeeModels"] == []:
            continue
        extracted_info.append(info)

    return extracted_info

# Example usage with your provided string
definitions_str = """your long string here"""
info = extract_info(content)
info


{Definition}
Extend lines are:  []
{deviceEndpoints, light}
Extend lines are:  []
{
        zigbeeModel: ['DimmerSwitch-2Gang-ZB3.0'],
        model: 'D086-ZG',
        vendor: 'HZC Electric',
        description: 'Zigbee dual dimmer',
        extend: [
            deviceEndpoints({endpoints: {'l1': 1, 'l2': 2}
Extend lines are:  []
{endpointNames: ['l1', 'l2'], configureReporting: true}
Extend lines are:  []
{
        zigbeeModel: ['TempAndHumSensor-ZB3.0'],
        model: 'S093TH-ZG',
        vendor: 'HZC Electric',
        description: 'Temperature and humidity sensor',
        fromZigbee: [fz.temperature, fz.humidity, fz.linkquality_from_basic],
        toZigbee: [],
        exposes: [e.temperature(), e.humidity()], // Unfortunately, battery percentage is not reported by this device
    }
Extend lines are:  []


[{'zigbeeModels': ["['DimmerSwitch-2Gang-ZB3.0']"],
  'fromZigbee': [],
  'toZigbee': [],
  'extend': []},
 {'zigbeeModels': ["['TempAndHumSensor-ZB3.0']"],
  'fromZigbee': ['fz.temperature', 'fz.humidity', 'fz.linkquality_from_basic'],
  'toZigbee': [],
  'extend': []}]

In [40]:
import os

folder_path = "./zigbee-herdsman-converters/src/devices/"

# Iterate over the files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file is a document
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            content = file.read()
        
        # Call the extract_info function on the document content
        info = extract_info(content)
        
        # Do something with the extracted info
        print(info)


[{'zigbeeModels': ["['LUBEEZ-12AB']"], 'fromZigbee': [], 'toZigbee': [], 'extend': []}]
[{'zigbeeModels': ["['DCH-B112']"], 'fromZigbee': ['fz.battery'], 'toZigbee': [], 'extend': []}]
[{'zigbeeModels': ["['X2SK11']"], 'fromZigbee': [], 'toZigbee': [], 'extend': ['onOff']}]
[{'zigbeeModels': ["['ABL-LIGHT-Z-001']"], 'fromZigbee': [], 'toZigbee': [], 'extend': ['light']}, {'zigbeeModels': ["['ABL-LIGHT-Z-201']"], 'fromZigbee': [], 'toZigbee': [], 'extend': ['light']}, {'zigbeeModels': ["['ABL-LIGHT-Z-202']"], 'fromZigbee': [], 'toZigbee': [], 'extend': ['light']}]
[{'zigbeeModels': ["['SZ-ESW01']"], 'fromZigbee': ['fz.on_off', 'fz.metering'], 'toZigbee': ['tz.on_off'], 'extend': []}, {'zigbeeModels': ["['SZ-ESW01-AU']"], 'fromZigbee': ['fz.on_off', 'fz.metering', 'fz.electrical_measurement'], 'toZigbee': ['tz.on_off'], 'extend': []}, {'zigbeeModels': ["['SZ-ESW02']"], 'fromZigbee': ['fz.on_off', 'fz.metering'], 'toZigbee': ['tz.on_off'], 'extend': []}, {'zigbeeModels': ["['XHS2-SE']"], 

In [15]:
pattern = r'((\w+\.fz\.\w+)|(fz\.\w+))'
matches = re.findall(pattern, content)

print(matches)
pattern = r'.*extend.*'
extend_lines = re.findall(pattern, content)

print(extend_lines)
function_pattern = r'(\w+)\('

for line in extend_lines:
    function_calls = re.findall(function_pattern, line)
    print(function_calls)
    
def get_fz_functions(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    pattern = r'fz\.\w+'
    matches = re.findall(pattern, content)
    return matches



def get_extend_lines(file_path):
    res = []
    with open(file_path, 'r') as file:
        content = file.read()
    pattern = r'.*extend.*'
    extend_lines = re.findall(pattern, content)
    function_pattern = r'(\w+)\('

    for line in extend_lines:
        function_calls = re.findall(function_pattern, line)
        print(function_calls)
        
    return extend_lines


[('fz.battery', '', 'fz.battery'), ('fz.command_toggle', '', 'fz.command_toggle'), ('fz.command_recall', '', 'fz.command_recall'), ('fz.on_off', '', 'fz.on_off'), ('fz.electrical_measurement', '', 'fz.electrical_measurement'), ('fz.command_on', '', 'fz.command_on'), ('fz.command_off', '', 'fz.command_off'), ('legacy.fz.eria_81825_updown', 'legacy.fz.eria_81825_updown', '')]
['        extend: [light({colorTemp: {range: undefined}, color: {applyRedFix: true}})],', "        extend: [light({colorTemp: {range: [153, 500]}, color: {modes: ['xy', 'hs'], applyRedFix: true}})],", '        extend: [light({colorTemp: {range: [153, 500]}, color: true})],', '        extend: [light()],', '        extend: [light({configureReporting: true})],', '        extend: [light({configureReporting: true})],', '        extend: [onOff()],', "        extend: [light({colorTemp: {range: [153, 500]}, color: {modes: ['xy', 'hs'], applyRedFix: true}})],", "        extend: [light({colorTemp: {range: [153, 500]}, color: 